# Beyond Bluetooth

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ..

/home/suriya/dcu/Group-Project


## Make Data 

In [50]:
from src.features.beyond import sensor_summary
from src.features.common import postproc_default
from src.dataset.tc4tl import make_datasets

trainset, devset, testset = make_datasets(sensor_summary, postproc_default, verbose=True,
                                          testing=0, tunables={})

100%|██████████████████████████████████████| 8423/8423 [00:37<00:00, 222.63it/s]


In [51]:
trainset

,RssiMin,RssiPercentile:1,RssiPercentile:5,RssiPercentile:10,RssiPercentile:25,RssiPercentile:40,RssiMean,RssiPercentile:65,RssiPercentile:80,RssiPercentile:90,RssiPercentile:95,RssiPercentile:99,RssiMax,Accelerometer_x:Min,Accelerometer_x:Mean,Accelerometer_x:Max,Accelerometer_y:Min,Accelerometer_y:Mean,Accelerometer_y:Max,Accelerometer_z:Min,Accelerometer_z:Mean,Accelerometer_z:Max,Gyroscope_x:Min,Gyroscope_x:Mean,Gyroscope_x:Max,Gyroscope_y:Min,Gyroscope_y:Mean,Gyroscope_y:Max,Gyroscope_z:Min,Gyroscope_z:Mean,Gyroscope_z:Max,Attitude_x:Min,Attitude_x:Mean,Attitude_x:Max,Attitude_y:Min,Attitude_y:Mean,Attitude_y:Max,Attitude_z:Min,Attitude_z:Mean,Attitude_z:Max,...,Gravity_z:Min,Gravity_z:Mean,Gravity_z:Max,Magnetic-field_x:Min,Magnetic-field_x:Mean,Magnetic-field_x:Max,Magnetic-field_y:Min,Magnetic-field_y:Mean,Magnetic-field_y:Max,Magnetic-field_z:Min,Magnetic-field_z:Mean,Magnetic-field_z:Max,Altitude_x:Min,Altitude_x:Mean,Altitude_x:Max,Altitude_y:Min,Altitude_y:Mean,Altitude_y:Max,Heading_x1:Min,Heading_x1:Mean,Heading_x1:Max,Heading_y1:Min,Heading_y1:Mean,Heading_y1:Max,Heading_z1:Min,Heading_z1:Mean,Heading_z1:Max,Heading_x2:Min,Heading_x2:Mean,Heading_x2:Max,Heading_y2:Min,Heading_y2:Mean,Heading_y2:Max,Heading_z2:Min,Heading_z2:Mean,Heading_z2:Max,CoarseGrain,fileid,Distance,DistanceFloat
0,-88.0,-79.00,-75.0,-74.0,-67.00,-64.0,-62.133482,-59.0,-54.0,-52.0,-52.0,-50.0,-49.0,-0.189240,0.024843,0.452316,-0.932861,-0.655037,-0.384369,-0.973267,-0.717481,-0.376022,-0.325451,0.007361,0.718535,-0.595014,-0.001176,0.427516,-0.149214,0.009136,0.594563,0.388814,0.726321,1.165844,-0.128704,0.028797,0.506453,1.809946,1.955215,2.213493,...,-0.917706,-0.725391,-0.393248,-15.442633,-7.255525,21.910954,-58.441452,-51.740549,-39.459232,-41.983582,-21.832841,0.421722,-0.864037,-0.054690,0.179680,99.976196,99.983530,99.986435,134.498199,140.964962,145.346619,145.082352,151.549114,155.930771,15.201146,15.225778,15.290328,-15.141745,-12.774888,-10.012350,-54.842712,-43.990857,-39.718319,-36.270050,-30.593178,-14.488922,0,aaadbuvp_tc4tl20.csv,1.8,1.8
1,-76.0,-76.00,-64.0,-60.0,-57.00,-56.0,-55.255556,-54.0,-50.0,-49.0,-48.0,-48.0,-48.0,-0.226959,0.187249,0.460770,0.277084,0.403910,0.751709,-0.929611,-0.013802,1.077881,-0.398142,-0.017030,0.227942,-1.020782,-0.019209,0.218994,-0.451402,0.015339,1.019249,-0.493580,-0.410232,-0.275184,-0.262122,1.313448,2.797158,-1.490315,0.126241,1.613801,...,-0.928316,-0.022922,0.868001,-3.491909,8.966918,25.032932,-0.270653,17.076906,38.654633,-49.587830,1.615656,44.842403,-0.332932,-0.066388,0.042282,102.566574,102.568475,102.569412,257.874786,280.976462,341.449677,272.185120,295.286795,355.760010,13.083611,13.094880,13.109203,-6.523527,-0.660559,9.231770,22.951851,25.961174,38.456741,-32.853706,31.900434,44.096672,1,aaamkcii_tc4tl20.csv,3.0,3.0
2,-89.0,-80.00,-76.6,-75.0,-69.00,-63.0,-62.289089,-57.0,-54.0,-52.0,-51.0,-47.0,-47.0,0.017563,0.050939,0.098373,-0.837662,-0.816428,-0.784485,-0.614807,-0.577063,-0.539825,-0.102149,0.007156,0.116281,-0.183989,-0.005191,0.084268,-0.062263,-0.006214,0.039732,0.917538,0.954430,0.984308,0.040087,0.087616,0.149159,-1.527429,1.001973,3.063332,...,-0.601385,-0.575466,-0.551723,-20.526367,-2.741884,19.228935,-52.590126,-38.768912,-27.756584,-41.024353,-23.775794,-5.399658,-0.153255,-0.002921,0.103050,101.015083,101.019933,101.022141,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,aabqtowt_tc4tl20.csv,1.2,1.2
3,-93.0,-87.00,-83.0,-78.0,-71.00,-68.0,-67.938259,-64.0,-62.0,-61.0,-59.0,-56.0,-53.0,-0.187805,-0.067156,0.390930,-1.110733,-0.965587,-0.496796,-0.817169,0.148694,0.256027,-0.173978,0.028015,0.801699,-0.397326,0.053077,3.652168,-0.337040,0.044304,2.019889,0.711186,1.337362,1.554614,-2.937387,-2.012155,2.788117,-3.125900,-1.171815,3.099751,...,-0.731285,0.147047,0.246787,-32.063553,1.718974,31.522234,-39.879414,-31.338613,-14.004144,-39.613647,12.

In [52]:
predictors = []
for col in trainset.columns:
    if "Rssi" in col or "Coarse" in col or "_" in col:
        predictors.append(col)
predictors

['RssiMin',
 'RssiPercentile:1',
 'RssiPercentile:5',
 'RssiPercentile:10',
 'RssiPercentile:25',
 'RssiPercentile:40',
 'RssiMean',
 'RssiPercentile:65',
 'RssiPercentile:80',
 'RssiPercentile:90',
 'RssiPercentile:95',
 'RssiPercentile:99',
 'RssiMax',
 'Accelerometer_x:Min',
 'Accelerometer_x:Mean',
 'Accelerometer_x:Max',
 'Accelerometer_y:Min',
 'Accelerometer_y:Mean',
 'Accelerometer_y:Max',
 'Accelerometer_z:Min',
 'Accelerometer_z:Mean',
 'Accelerometer_z:Max',
 'Gyroscope_x:Min',
 'Gyroscope_x:Mean',
 'Gyroscope_x:Max',
 'Gyroscope_y:Min',
 'Gyroscope_y:Mean',
 'Gyroscope_y:Max',
 'Gyroscope_z:Min',
 'Gyroscope_z:Mean',
 'Gyroscope_z:Max',
 'Attitude_x:Min',
 'Attitude_x:Mean',
 'Attitude_x:Max',
 'Attitude_y:Min',
 'Attitude_y:Mean',
 'Attitude_y:Max',
 'Attitude_z:Min',
 'Attitude_z:Mean',
 'Attitude_z:Max',
 'Gravity_x:Min',
 'Gravity_x:Mean',
 'Gravity_x:Max',
 'Gravity_y:Min',
 'Gravity_y:Mean',
 'Gravity_y:Max',
 'Gravity_z:Min',
 'Gravity_z:Mean',
 'Gravity_z:Max',
 'Ma

## Evaluate

In [53]:
from src.models.evaluate import dual_evaluation
import pandas as pd
from src.ndcf import compute_ndcf

report, sysout = dual_evaluation(pd.concat([trainset, devset]), testset,
                    predictors=predictors,
                    target="Distance")
ndcf = compute_ndcf(sysout)
ndcf["nDCF"].mean()

0.09807007696040455

In [56]:
ndcf

,Subset,Distance,P_miss,P_fa,nDCF
0,fine_grain,1.2,0.090263,0.014074,0.104337
1,fine_grain,1.8,0.065009,0.021552,0.086561
2,fine_grain,3.0,0.033333,0.068620,0.101954
3,coarse_grain,1.8,0.077301,0.022127,0.099429


In [54]:
print(report["cg=1"])

              precision    recall  f1-score   support

         1.2       0.96      0.91      0.93      1407
         1.8       0.93      0.90      0.91      1408
         3.0       0.90      0.95      0.92      1385
         4.5       0.90      0.93      0.92      1399

    accuracy                           0.92      5599
   macro avg       0.92      0.92      0.92      5599
weighted avg       0.92      0.92      0.92      5599



In [55]:
print(report["cg=0"])

              precision    recall  f1-score   support

         1.8       0.98      0.92      0.95      1423
         4.5       0.93      0.98      0.95      1401

    accuracy                           0.95      2824
   macro avg       0.95      0.95      0.95      2824
weighted avg       0.95      0.95      0.95      2824



## Combine with Categorical Features

In [64]:
from src.features.common import postproc_categorical, read_non_sensor_data
from src.featutils import combine
import category_encoders as ce

trainset, devset, testset = make_datasets(combine(sensor_summary, read_non_sensor_data),
                                          postproc_categorical, verbose=True, testing=0,
                                          tunables={ 
                                              "CategoricalEncoder" : ce.polynomial.PolynomialEncoder
                                          })

100%|████████████████████████████████████| 15552/15552 [01:09<00:00, 222.90it/s]


NO Pipe input given!


100%|████████████████████████████████████████| 935/935 [00:04<00:00, 233.74it/s]


Pipe input given!


100%|██████████████████████████████████████| 8423/8423 [00:34<00:00, 243.41it/s]


Pipe input given!


In [65]:
trainset

,RssiMin,RssiPercentile:1,RssiPercentile:5,RssiPercentile:10,RssiPercentile:25,RssiPercentile:40,RssiMean,RssiPercentile:65,RssiPercentile:80,RssiPercentile:90,RssiPercentile:95,RssiPercentile:99,RssiMax,Accelerometer_x:Min,Accelerometer_x:Mean,Accelerometer_x:Max,Accelerometer_y:Min,Accelerometer_y:Mean,Accelerometer_y:Max,Accelerometer_z:Min,Accelerometer_z:Mean,Accelerometer_z:Max,Gyroscope_x:Min,Gyroscope_x:Mean,Gyroscope_x:Max,Gyroscope_y:Min,Gyroscope_y:Mean,Gyroscope_y:Max,Gyroscope_z:Min,Gyroscope_z:Mean,Gyroscope_z:Max,Attitude_x:Min,Attitude_x:Mean,Attitude_x:Max,Attitude_y:Min,Attitude_y:Mean,Attitude_y:Max,Attitude_z:Min,Attitude_z:Mean,Attitude_z:Max,...,fileid,Distance,DistanceFloat,Cat:TXDevice,Cat:TXPower,Cat:RXDevice,Cat:TXCarry,Cat:RXCarry,Cat:RXPose,Cat:TXPose,intercept,Cat:TXDevice_0,Cat:TXDevice_1,Cat:TXDevice_2,Cat:TXDevice_3,Cat:TXDevice_4,Cat:TXDevice_5,Cat:TXDevice_6,Cat:TXDevice_7,Cat:TXDevice_8,Cat:TXDevice_9,Cat:TXDevice_10,Cat:TXDevice_11,Cat:TXDevice_12,Cat:TXDevice_13,Cat:TXPower_0,Cat:TXPower_1,Cat:RXDevice_0,Cat:RXDevice_1,Cat:RXDevice_2,Cat:RXDevice_3,Cat:RXDevice_4,Cat:RXDevice_5,Cat:RXDevice_6,Cat:RXDevice_7,Cat:RXDevice_8,Cat:TXCarry_0,Cat:RXCarry_0,Cat:RXPose_0,Cat:TXPose_0
0,-88.0,-79.00,-75.0,-74.0,-67.00,-64.0,-62.133482,-59.0,-54.0,-52.0,-52.0,-50.0,-49.0,-0.189240,0.024843,0.452316,-0.932861,-0.655037,-0.384369,-0.973267,-0.717481,-0.376022,-0.325451,0.007361,0.718535,-0.595014,-0.001176,0.427516,-0.149214,0.009136,0.594563,0.388814,0.726321,1.165844,-0.128704,0.028797,0.506453,1.809946,1.955215,2.213493,...,aaadbuvp_tc4tl20.csv,1.8,1.8,iPhoneX,12,iPhone8,pocket,hand,standing,standing,1,-0.418330,0.472270,-0.456256,0.393641,-0.307724,0.219002,-0.141859,0.083314,-0.044039,0.020706,-0.008499,0.002954,-0.000820,0.000158,-7.071068e-01,0.408248,-0.495434,0.522233,-0.453425,0.336581,-0.214834,0.116775,-0.052694,0.018699,-0.004535,-0.707107,-0.707107,-0.707107,-0.707107
1,-76.0,-76.00,-64.0,-60.0,-57.00,-56.0,-55.255556,-54.0,-50.0,-49.0,-48.0,-48.0,-48.0,-0.226959,0.187249,0.460770,0.277084,0.403910,0.751709,-0.929611,-0.013802,1.077881,-0.398142,-0.017030,0.227942,-1.020782,-0.019209,0.218994,-0.451402,0.015339,1.019249,-0.493580,-0.410232,-0.275184,-0.262122,1.313448,2.797158,-1.490315,0.126241,1.613801,...,aaamkcii_tc4tl20.csv,3.0,3.0,iPhone11,8,iPhoneXS,pocket,pocket,standing,standing,1,-0.358569,0.269869,-0.065179,-0.168703,0.351684,-0.438004,0.425576,-0.345159,0.239071,-0.141982,0.071638,-0.029959,0.009845,-0.002210,-5.551115e-17,-0.816497,-0.385337,0.174078,0.151142,-0.411377,0.501280,-0.428174,0.275179,-0.130893,0.040816,-0.707107,0.707107,-0.707107,-0.707107
2,-89.0,-80.00,-76.6,-75.0,-69.00,-63.0,-62.289089,-57.0,-54.0,-52.0,-51.0,-47.0,-47.0,0.017563,0.050939,0.098373,-0.837662,-0.816428,-0.784485,-0.614807,-0.577063,-0.539825,-0.102149,0.007156,0.116281,-0.183989,-0.005191,0.084268,-0.062263,-0.006214,0.039732,0.917538,0.954430,0.984308,0.040087,0.087616,0.149159,-1.527429,1.001973,3.063332,...,aabqtowt_tc4tl20.csv,1.2,1.2,iPhone6Plus,12,iPhone11,hand,hand,standing,standing,1,-0.298807,0.098606,0.175483,-0.341733,0.300960,-0.084231,-0.185507,0.379950,-0.436039,0.371110,-0.248910,0.132074,-0.053325,0.014367,-7.071068e-01,0.408248,-0.275241,-0.087039,0.377854,-0.317882,-0.035806,0.389249,-0.503518,0.373979,-0.163266,0.707107,-0.707107,-0.707107,-0.707107
3,-93.0,-87.00,-83.0,-78.0,-71.00,-68.0,-67.938259,-64.0,-62.0,-61.0,-59.0,-56.0,-53.0,-0.187805,-0.067156,0.390930,-1.110733,-0.965587,-0.496796,-0.817169,0.148694,0.256027,-0.173978,0.028015,0.801699,-0.397326,0.053077,3.652168,-0.337040,0.044304,2.019889,0.711186,1.337362,1.554614,-2.937387,-2.012155,2.788117,-3.125900,-1.171815,3.099751,...,aadkjwss_tc4tl20.csv,1.8,1.8,iPhone7,12,iPhone11,pocket,pocket,sitting,sitting,1,-0.239046,-0.041518,0.290801,-0.276847,0.013526,0.269541,-0.338278,0.143740,0.166479,-0.392270,0.432254,-0.323223,0.170641,-0.057470,-7.071068e-01,0.408248,-0.275241,-0.087039,0.377854,-0.317882,-0.035806,0.389

In [66]:
predictors = []
for col in trainset.columns:
    if "Rssi" in col or "Coarse" in col or "_" in col:
        predictors.append(col)
predictors

['RssiMin',
 'RssiPercentile:1',
 'RssiPercentile:5',
 'RssiPercentile:10',
 'RssiPercentile:25',
 'RssiPercentile:40',
 'RssiMean',
 'RssiPercentile:65',
 'RssiPercentile:80',
 'RssiPercentile:90',
 'RssiPercentile:95',
 'RssiPercentile:99',
 'RssiMax',
 'Accelerometer_x:Min',
 'Accelerometer_x:Mean',
 'Accelerometer_x:Max',
 'Accelerometer_y:Min',
 'Accelerometer_y:Mean',
 'Accelerometer_y:Max',
 'Accelerometer_z:Min',
 'Accelerometer_z:Mean',
 'Accelerometer_z:Max',
 'Gyroscope_x:Min',
 'Gyroscope_x:Mean',
 'Gyroscope_x:Max',
 'Gyroscope_y:Min',
 'Gyroscope_y:Mean',
 'Gyroscope_y:Max',
 'Gyroscope_z:Min',
 'Gyroscope_z:Mean',
 'Gyroscope_z:Max',
 'Attitude_x:Min',
 'Attitude_x:Mean',
 'Attitude_x:Max',
 'Attitude_y:Min',
 'Attitude_y:Mean',
 'Attitude_y:Max',
 'Attitude_z:Min',
 'Attitude_z:Mean',
 'Attitude_z:Max',
 'Gravity_x:Min',
 'Gravity_x:Mean',
 'Gravity_x:Max',
 'Gravity_y:Min',
 'Gravity_y:Mean',
 'Gravity_y:Max',
 'Gravity_z:Min',
 'Gravity_z:Mean',
 'Gravity_z:Max',
 'Ma

In [67]:
from src.models.evaluate import dual_evaluation
import pandas as pd
from src.ndcf import compute_ndcf

report, sysout = dual_evaluation(pd.concat([trainset, devset]), testset,
                    predictors=predictors,
                    target="Distance")
ndcf = compute_ndcf(sysout)
ndcf["nDCF"].mean()

0.091341887053001

In [68]:
print(report["cg=1"])

              precision    recall  f1-score   support

         1.2       0.97      0.91      0.94      1407
         1.8       0.94      0.91      0.93      1408
         3.0       0.92      0.95      0.93      1385
         4.5       0.91      0.96      0.93      1399

    accuracy                           0.93      5599
   macro avg       0.93      0.93      0.93      5599
weighted avg       0.93      0.93      0.93      5599



In [69]:
print(report["cg=0"])

              precision    recall  f1-score   support

         1.8       0.98      0.91      0.94      1423
         4.5       0.91      0.98      0.94      1401

    accuracy                           0.94      2824
   macro avg       0.94      0.94      0.94      2824
weighted avg       0.94      0.94      0.94      2824

